* 이 글은 Hands-On ML(2판)의 실습을 정리한 것입니다.

# [ DataMining ] 지도학습 - 앙상블 학습

## 1. 데이터 로드

In [3]:
from sklearn.datasets import make_moons
x, y = make_moons(n_samples=100, noise=0.15)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=30)

## 2. 사이킷런의 투표 기반 분류기(VotingClassifier) 만들고 훈련

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()            # 로지스틱 regression
rnd_clf = RandomForestClassifier()        # 랜덤포레스트
svm_clf = SVC()                           # SVC

voting_clf = VotingClassifier(estimators=[('lr',log_clf), ('rf',rnd_clf), ('svd',svm_clf)], 
                              voting='hard')
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svd', SVC())])

## - 분류기의 테스트셋 정확도를 확인 

In [6]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.9333333333333333
SVC 0.9
VotingClassifier 0.9


=> VotingClassifier(투표 기반 분류기)가 다른 개별 분류기보다 성능이 조금 더 높음

## 3. Decision Tree 분류기 500개의 앙상블 훈련

#### - 각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련(배깅의 경우)

In [12]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

'''BaggingClassifier: 기반이 되는 분류기가 결정 트리 분류기처럼 클래스 확률을 추정'''
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            bootstrap=True, n_jobs=-1) #max_samples=100을 넣으니 실행 안됨
# predict_proba(): 함수를 넣으면 직접 투포 대신 간접투표 방식을 사용
bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)